In [1]:
import torch
import torchvision
import torch.nn as nn
from torchvision.models import VGG16_Weights

# 测试数据集读取
## 自定义数据集

In [3]:
from loadData import load_data
train_loader, test_loader, database_loader, train_num, test_num, database_num= load_data(250,4)

Files already downloaded and verified
train_dataset 50000
test_dataset 10000
database_dataset 10000
划分后train_dataset 50000
划分后test_dataset 10000
划分后database_dataset 50000


In [11]:
k = 0
for img ,cls, i in train_loader:
  if k == 3 :
    break
  print("索引:",i)
  print("图像:",img)
  print("类别",cls)
  print("索引shape:",i.shape)
  print("图像shape:",img.shape)
  print("类别shape:",cls.shape)
  k = k+1

索引: tensor([45551, 47879, 27007,  6802,  7026,  9705, 17951, 24817,  4233, 35583,
        25490, 32287, 25501, 36654, 19097, 38050, 12021, 12316, 49738, 13537,
        47244, 25444, 36763, 30778, 14147, 20284, 17486,  7989, 37191, 44754,
        49472, 19633, 13542, 33630,  4966, 29463,  4090, 30270, 33944, 30564,
        14836, 16287, 27826,  8560, 43842, 10528, 39054, 29526,  4664, 12761,
        48630, 30731,  6330, 48900, 35152, 11001,   993, 30273, 26595, 11979,
        33635, 15795, 41101,  6197, 29386, 44940, 29763, 25040, 33155, 38413,
        27314, 24102, 16632, 18295, 24160, 43678, 25223, 38922, 38183, 32320,
         9999, 19468, 47069, 43143,   580,  6933, 26184, 35301, 47014, 44472,
        22948,  1028, 48393, 42772, 16800, 21328, 43995, 26295, 30074, 44662,
         9592, 20355, 45864, 34638, 17160, 32541, 39219, 12033,  9917, 28446,
        25282, 12355, 21096,  1803, 47530, 30993, 31766, 20074, 34228, 45688,
        35677, 24216, 39140,  5799,  3838, 21856, 35149, 113

## pytorch内置数据集

In [16]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    # transforms.Resize(args.crop_size),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_ds = datasets.CIFAR10(
  root="dataset/cifar",
  train=True,
  download=False,
  transform=transform,
)
test_ds = datasets.CIFAR10(
  root="dataset/cifar",
  train=False,
  download=False,
  transform=transform,
  target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

In [17]:
train_dataloader = DataLoader(train_ds, batch_size=250, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=250, shuffle=False)

In [18]:
k=0
for img ,cls, i in train_loader:
  if k == 3 :
    break
  print("索引:",i)
  print("图像:",img)
  print("类别",cls)
  print("索引shape:",i.shape)
  print("图像shape:",img.shape)
  print("类别shape:",cls.shape)
  k = k+1

索引: tensor([38704,  4208, 36667, 48153, 13591, 43891,  8730,  1204, 44386,  3254,
        36458, 45825, 14427, 28404,  7381, 42207, 33917, 44880, 36606, 15333,
        44628, 32877, 37943, 22500, 42101, 47862, 37718, 30597,  3264, 46677,
        17466, 30600,   619, 10863, 16665, 33925, 44165, 21366, 46032, 35883,
        34302,  2119, 18002, 21350, 36311, 17449, 36888, 26587,  7192, 15958,
        33305, 12115, 39954, 15936, 23145, 34914,  6086, 48018,  3539, 30750,
         8759, 47864, 48312, 30719, 36172, 16761, 38205, 18299, 45769,  6231,
        47394, 19833, 17132, 34320, 15472, 24483,  8878, 27365, 49198, 15582,
        17113,  2926, 18311,  4104, 13388, 36650,  8635, 21633, 46927, 24103,
        39544, 23183, 36712, 37506,  5387, 34404, 29499, 34970, 19997, 33954,
        13375,  4664, 40402,  6809, 49210, 43003, 34443, 49451,  4749,  8311,
        33684, 16296, 24755, 36877, 30739, 30643, 34336, 16943, 26447, 25368,
         4222, 30448, 35034, 47052,  4527,  5067, 13565, 392